# Exploring Hacker News Post
In this project, we'll work with a dataset of submissions to popular technology site [Hacker News](https://news.ycombinator.com/). Hacker News is a site where user-submitted stories (known as "posts") receive votes and comments, similar to reddit.  We're specifically interested in analysing posts with titles that begin with either;

- **Ask HN** :- where users submit the posts using the tag to ask the community a specific question. Or;

- **Show HN** :- where users submit posts to show the community a project, or just something interesting.

The whole dataset can be found [here](https://www.kaggle.com/datasets/hacker-news/hacker-news-posts), but we have reduced the length of the dataset to approximately 20,000 rows by removing all submissions that didn't receive any comments.

The aim of of analysis is to dertmine:
1. which posts, amont the two(AskHN and ShowHN) receive more comments on average.
2. The appropriate time(hours) to create a post with high chances of receiving comments.


### Reading in the Dataset
We'll start by importing, opening and reading in the data

In [1]:
#import the data and explore the first five rows

from csv import reader

opened_file = open('hacker_news.csv')
read_file = reader(opened_file)
hn = list(read_file)
print(hn[:5])

[['id', 'title', 'url', 'num_points', 'num_comments', 'author', 'created_at'], ['12224879', 'Interactive Dynamic Video', 'http://www.interactivedynamicvideo.com/', '386', '52', 'ne0phyte', '8/4/2016 11:52'], ['10975351', 'How to Use Open Source and Shut the Fuck Up at the Same Time', 'http://hueniverse.com/2016/01/26/how-to-use-open-source-and-shut-the-fuck-up-at-the-same-time/', '39', '10', 'josep2', '1/26/2016 19:30'], ['11964716', "Florida DJs May Face Felony for April Fools' Water Joke", 'http://www.thewire.com/entertainment/2013/04/florida-djs-april-fools-water-joke/63798/', '2', '1', 'vezycash', '6/23/2016 22:20'], ['11919867', 'Technology ventures: From Idea to Enterprise', 'https://www.amazon.com/Technology-Ventures-Enterprise-Thomas-Byers/dp/0073523429', '3', '1', 'hswarna', '6/17/2016 0:01']]


In [2]:
#Removing headers from a list of Lists

opened_file = open('hacker_news.csv')
read_file = reader(opened_file)
hn = list(read_file)
headers = hn[:1]
hn = hn[1:]
print(headers)
print('\n')
print(hn[:5])


[['id', 'title', 'url', 'num_points', 'num_comments', 'author', 'created_at']]


[['12224879', 'Interactive Dynamic Video', 'http://www.interactivedynamicvideo.com/', '386', '52', 'ne0phyte', '8/4/2016 11:52'], ['10975351', 'How to Use Open Source and Shut the Fuck Up at the Same Time', 'http://hueniverse.com/2016/01/26/how-to-use-open-source-and-shut-the-fuck-up-at-the-same-time/', '39', '10', 'josep2', '1/26/2016 19:30'], ['11964716', "Florida DJs May Face Felony for April Fools' Water Joke", 'http://www.thewire.com/entertainment/2013/04/florida-djs-april-fools-water-joke/63798/', '2', '1', 'vezycash', '6/23/2016 22:20'], ['11919867', 'Technology ventures: From Idea to Enterprise', 'https://www.amazon.com/Technology-Ventures-Enterprise-Thomas-Byers/dp/0073523429', '3', '1', 'hswarna', '6/17/2016 0:01'], ['10301696', 'Note by Note: The Making of Steinway L1037 (2007)', 'http://www.nytimes.com/2007/11/07/movies/07stein.html?_r=0', '8', '2', 'walterbell', '9/30/2015 4:12']]


The table below highlights the descriptions of the table with the contents therein

| Column | Description |
|--------|-------------|
|"id"    | Hacker News unique identifier|
|"title" | Title of the Post|
|" url"  | URL link to the post|
|"num_points" | Number of points the post acquired|
|"num_comments" | Number of comments on the post|
|"author"| Username of the person who submitted the post|
|"created_at" | Date and time of the post's submission|


### Extracting Ask HN and Show HN Posts
- Since we're only concerned with post titles beginning with Ask HN or Show HN, we'll create new lists of lists containing just the data for those titles.
- We'll use the string method `startwith` to find the posts that begin either with **Ask HN** or **Show HN**


In [3]:
ask_posts = []
show_posts = []
other_posts = []

for post in hn:
    title = post[1]
    
    if title.lower().startswith("ask hn"):
        ask_posts.append(post)
        
    elif title.lower().startswith("show hn"):
        show_posts.append(post)
        
    else:
        other_posts.append(post)
        
print('No. of ask_posts:' , str(len(ask_posts)))
print('No. of show_posts:' , str(len(show_posts)))
print('No. of other_posts:' , str(len(other_posts)))
     

No. of ask_posts: 1744
No. of show_posts: 1162
No. of other_posts: 17194


### Average No. of Comments for Ask HN and Show HN Posts

The specific objective of this analysis to focus on the post that receives more comments on average hence  between the two posts, we'll calculate the average and proceed with the post that has a higher average value.

In [4]:
##Average no. of comments for Ask HN

total_ask_comments = 0

for post in ask_posts:
    num_comment = int(post[4])
    total_ask_comments += num_comment

avg_ask_comments = total_ask_comments/len(ask_posts)

#convert the average value to precisionof two
#decimal placesusing the string.format() method
print('{:.2f}'.format(avg_ask_comments)) 

14.04


In [5]:
##Average no. of comments for Show HN

total_show_comments = 0

for post in show_posts:
    num_comment = int(post[4])
    total_show_comments += num_comment

avg_show_comments = total_show_comments/len(show_posts)

print('{:.2f}'.format(avg_show_comments))
    

10.32


As per the illustrations above, the figures demonstrates that **Ask HN** averages ```4 comments``` more than the __Show HN__ at any particular occurance.

### Finding the Number of Ask Posts and Comments by Hour Created

- Our objective is to determine if ask posts created at a certain _time_ are more likeliy to attract comments.

In [39]:
## import the datetime module
import datetime as dt

result_list = []

for post in ask_posts:
    created_at = post[6]
    num_comments = int(post[4])
    result_list.append([created_at, num_comments])
    
#Create two empty dictionaries
#loop through the result_list
#parse the date and create datetime object
#select the hour using strftime method
#create dictionaries for the counts_by_hour and comments_by_hour
counts_by_hour = {}
comments_by_hour = {}

for row in result_list:
    dt_str = row[0] #exxtract hour from the date
    num_comments = row[1]
    dt_obj = dt.datetime.strptime(dt_str, '%m/%d/%Y %H:%M')
    hr_str = dt_obj.strftime('%H')
    
    if hr_str not in counts_by_hour:
        counts_by_hour[hr_str] = 1
        comments_by_hour[hr_str] = num_comments
    else:
        counts_by_hour[hr_str] += 1
        comments_by_hour[hr_str] += num_comments
        
comments_by_hour
    

{'09': 251,
 '13': 1253,
 '10': 793,
 '14': 1416,
 '16': 1814,
 '23': 543,
 '12': 687,
 '17': 1146,
 '15': 4477,
 '21': 1745,
 '20': 1722,
 '02': 1381,
 '18': 1439,
 '03': 421,
 '05': 464,
 '19': 1188,
 '01': 683,
 '22': 479,
 '08': 492,
 '04': 337,
 '00': 447,
 '06': 397,
 '07': 267,
 '11': 641}

### Calculating the Average Number of Comments by Hour
We will use the **counts_by_hour** and **comments_by_hour** to calculate the average number of comments for _Ask HN_ posts by hour

In [7]:
avg_by_hour = []

for hr in comments_by_hour:
    avg_by_hour.append([hr, comments_by_hour[hr]/counts_by_hour[hr]])

(avg_by_hour)

[['09', 5.5777777777777775],
 ['13', 14.741176470588234],
 ['10', 13.440677966101696],
 ['14', 13.233644859813085],
 ['16', 16.796296296296298],
 ['23', 7.985294117647059],
 ['12', 9.41095890410959],
 ['17', 11.46],
 ['15', 38.5948275862069],
 ['21', 16.009174311926607],
 ['20', 21.525],
 ['02', 23.810344827586206],
 ['18', 13.20183486238532],
 ['03', 7.796296296296297],
 ['05', 10.08695652173913],
 ['19', 10.8],
 ['01', 11.383333333333333],
 ['22', 6.746478873239437],
 ['08', 10.25],
 ['04', 7.170212765957447],
 ['00', 8.127272727272727],
 ['06', 9.022727272727273],
 ['07', 7.852941176470588],
 ['11', 11.051724137931034]]

### Sorting and Printing Values from a List of Lists
The format of the results we displayed above makes it difficult to identify the hours with the highest values. We will therefore sort the list of lists and print the five highest values in a formati that is easier to read and comprehend.

In [9]:
#create an empty list;swap_avg_by_hour
#swap the elements in the avg_by_hour list of lists
#append the swapped elements to the empty list; swap_avg
swap_avg_by_hour = []

for row in avg_by_hour:
    swap_avg_by_hour.append([row[1],row[0]])
swap_avg_by_hour

[[5.5777777777777775, '09'],
 [14.741176470588234, '13'],
 [13.440677966101696, '10'],
 [13.233644859813085, '14'],
 [16.796296296296298, '16'],
 [7.985294117647059, '23'],
 [9.41095890410959, '12'],
 [11.46, '17'],
 [38.5948275862069, '15'],
 [16.009174311926607, '21'],
 [21.525, '20'],
 [23.810344827586206, '02'],
 [13.20183486238532, '18'],
 [7.796296296296297, '03'],
 [10.08695652173913, '05'],
 [10.8, '19'],
 [11.383333333333333, '01'],
 [6.746478873239437, '22'],
 [10.25, '08'],
 [7.170212765957447, '04'],
 [8.127272727272727, '00'],
 [9.022727272727273, '06'],
 [7.852941176470588, '07'],
 [11.051724137931034, '11']]

In [13]:
#sort the swap_avg_by_hour in descending order
sorted_swap = sorted(swap_avg_by_hour, reverse=True)

sorted_swap


[[38.5948275862069, '15'],
 [23.810344827586206, '02'],
 [21.525, '20'],
 [16.796296296296298, '16'],
 [16.009174311926607, '21'],
 [14.741176470588234, '13'],
 [13.440677966101696, '10'],
 [13.233644859813085, '14'],
 [13.20183486238532, '18'],
 [11.46, '17'],
 [11.383333333333333, '01'],
 [11.051724137931034, '11'],
 [10.8, '19'],
 [10.25, '08'],
 [10.08695652173913, '05'],
 [9.41095890410959, '12'],
 [9.022727272727273, '06'],
 [8.127272727272727, '00'],
 [7.985294117647059, '23'],
 [7.852941176470588, '07'],
 [7.796296296296297, '03'],
 [7.170212765957447, '04'],
 [6.746478873239437, '22'],
 [5.5777777777777775, '09']]

In [38]:
#Print the Top 5 Hours for Ask Posts Comments

print('Top 5 Hours for Ask Posts Comments')
print('---------------------------------------')
for avg, hour in sorted_swap[:5]:    
    hour_dt = dt.datetime.strptime(hour, '%H')
    hour_str = hour_dt.strftime('%H:%M')
    print('{} : {:.2f} average comments per post'.format(hour_str, avg))
    
     

Top 5 Hours for Ask Posts Comments
---------------------------------------
15:00 : 38.59 average comments per post
02:00 : 23.81 average comments per post
20:00 : 21.52 average comments per post
16:00 : 16.80 average comments per post
21:00 : 16.01 average comments per post


From the findings, it appears that creating question posts at **3PM** in Hacker News is likely to receive more comments than any other time. Since I am located in Kenya, we adhere to EAT and therefore it will be appropriate to create a post at around **11PM** EAT.